In [6]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import re

# set the max columns to none
pd.set_option('display.max_columns', None)
# set the max columns to none
pd.set_option('display.max_rows', None)

In [7]:
file_observations = '../1_scraping/scraped_data/cleaned/observation_details_116_clean.csv'

# Load the data
halsbandparkiet = pd.read_csv(file_observations, header=0, index_col="observation_id")
print(f"observation count before dropna {halsbandparkiet.shape[0]}")

# drop observations with missing crucial data
halsbandparkiet = halsbandparkiet.dropna(subset = ['species_id', 'species_name', 'date', 'gps_coordinates']) 
print(f"observation count after dropna {halsbandparkiet.shape[0]}")

# clean columns
halsbandparkiet['date'] = pd.to_datetime(halsbandparkiet['date'], format="mixed", errors='coerce')
halsbandparkiet["species_id"] = halsbandparkiet["species_id"].astype(int) # int parsing

halsbandparkiet["individual_amount"] = halsbandparkiet["amount"].apply(lambda x: re.findall(r"\d+", x)[0]).astype(int) # only keep the amount and cast as int
halsbandparkiet["observation_amount"] = 1

# location needs to stay a float, because there are a lot of NaN values
# observations_hp["location_id"] = observations_hp["location_id"].astype(int) # int parsing
halsbandparkiet_without_location = halsbandparkiet[halsbandparkiet["location_id"].isnull()]
print(f"observations without location {halsbandparkiet_without_location.shape[0]}")

# Longitude, Latitude
halsbandparkiet["Latitude"] = halsbandparkiet["gps_coordinates"].apply(lambda x: x.split(', ')[0]) # NB
halsbandparkiet["Longitude"] = halsbandparkiet["gps_coordinates"].apply(lambda x: x.split(', ')[1]) # OL

print(f"halsbandparkiet bevat {halsbandparkiet.shape[0]} observaties")
halsbandparkiet.head(5)

observation count before dropna 96951
observation count after dropna 96551
observations without location 857
halsbandparkiet bevat 96551 observaties


,species_id,species_name,species_name_scientific,validation,gps_coordinates,accuracy,source,date,amount,life_stage,activity,location_id,location,observer_id,observer_name,counting_method,method,individual_amount,observation_amount,Latitude,Longitude
observation_id,,,,,,,,,,,,,,,,,,,,,
336301369,116,Halsbandparkiet,Psittacula krameri,Goedgekeurd (automatische validatie),"50.8585, 4.6717",33m,iObs,2024-12-31 16:45:00,1,onbekend,ter plaatse,30919.0,Heverlee - Egenhoven - Egenhoven Bos (VB),40236.0,Frederik Fluyt,onbekend,onbekend,1,1,50.8585,4.6717
336233189,116,Halsbandparkiet,Psittacula krameri,Goedgekeurd (automatische validatie),"50.8044, 4.6010",30m,Showcase,2024-12-31 16:00:00,1,onbekend,ter plaatse,30976.0,Loonbeek - Korenheide (VB),3528.0,Externe bron,geen,onbekend,1,1,50.8044,4.6010
336205268,116,Halsbandparkiet,Psittacula krameri,Goedgekeurd (automatische validatie),"50.8584, 4.2921",3m,ObsMapp,2024-12-31 15:29:00,6,onbekend,foeragerend,32596.0,Berchem-Ste-Agathe/St-Agatha-Berchem - Koninck...,72841.0,Sara Velghe,onbekend,gezien en gehoord,6,1,50.8584,4.2921
336194978,116,Halsbandparkiet,Psittacula krameri,Goedgekeurd (automatische validatie),"50.9315, 4.3675",13m,ObsIdentify,2024-12-31 15:21:00,1,onbekend,ter plaatse,23260.0,Grimbergen (VB),359678.0,Kristof Vynckier,geen,onbekend,1,1,50.9315,4.3675
336193018,116,Halsbandparkiet,Psittacula krameri,Goedgekeurd (automatische validatie),"50.7396, 3.3351",14m,Site,2024-12-31 14:57:00,4,onbekend,ter plaatse,27358.0,Kooigem (WV),201247.0,Lore Vander Plancken,geen,gezien,4,1,50.7396,3.3351


In [8]:
# create geopandas
gdf_halsbandparkiet = gpd.GeoDataFrame(halsbandparkiet, 
                                       geometry=gpd.points_from_xy(halsbandparkiet.Longitude,halsbandparkiet.Latitude),
                                       crs="EPSG:4326")
gdf_halsbandparkiet.head()

,species_id,species_name,species_name_scientific,validation,gps_coordinates,accuracy,source,date,amount,life_stage,activity,location_id,location,observer_id,observer_name,counting_method,method,individual_amount,observation_amount,Latitude,Longitude,geometry
observation_id,,,,,,,,,,,,,,,,,,,,,,
336301369,116,Halsbandparkiet,Psittacula krameri,Goedgekeurd (automatische validatie),"50.8585, 4.6717",33m,iObs,2024-12-31 16:45:00,1,onbekend,ter plaatse,30919.0,Heverlee - Egenhoven - Egenhoven Bos (VB),40236.0,Frederik Fluyt,onbekend,onbekend,1,1,50.8585,4.6717,POINT (4.6717 50.8585)
336233189,116,Halsbandparkiet,Psittacula krameri,Goedgekeurd (automatische validatie),"50.8044, 4.6010",30m,Showcase,2024-12-31 16:00:00,1,onbekend,ter plaatse,30976.0,Loonbeek - Korenheide (VB),3528.0,Externe bron,geen,onbekend,1,1,50.8044,4.6010,POINT (4.601 50.8044)
336205268,116,Halsbandparkiet,Psittacula krameri,Goedgekeurd (automatische validatie),"50.8584, 4.2921",3m,ObsMapp,2024-12-31 15:29:00,6,onbekend,foeragerend,32596.0,Berchem-Ste-Agathe/St-Agatha-Berchem - Koninck...,72841.0,Sara Velghe,onbekend,gezien en gehoord,6,1,50.8584,4.2921,POINT (4.2921 50.8584)
336194978,116,Halsbandparkiet,Psittacula krameri,Goedgekeurd (automatische validatie),"50.9315, 4.3675",13m,ObsIdentify,2024-12-31 15:21:00,1,onbekend,ter plaatse,23260.0,Grimbergen (VB),359678.0,Kristof Vynckier,geen,onbekend,1,1,50.9315,4.3675,POINT (4.3675 50.9315)
336193018,116,Halsbandparkiet,Psittacula krameri,Goedgekeurd (automatische validatie),"50.7396, 3.3351",14m,Site,2024-12-31 14:57:00,4,onbekend,ter plaatse,27358.0,Kooigem (WV),201247.0,Lore Vander Plancken,geen,gezien,4,1,50.7396,3.3351,POINT (3.3351 50.7396)


In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import ipywidgets as widgets
from ipywidgets import interact

# Example: Load region polygons
belgium = gpd.read_file('GEODATA/kontur_boundaries_BE_20230628.gpkg')
gemeenten = belgium[(belgium["osm_admin_level"] == '8')].copy()

def filter_by_year(gdf, year):
    return gdf[gdf['date'].dt.year == year]

def plot_heatmap(year):
    filtered_gdf = filter_by_year(gdf_halsbandparkiet, year)
    
    # Create a base plot with the region polygons
    base = gemeenten.plot(color='lightgray', edgecolor='black', figsize=(10, 10))
    
    # Plot the heatmap
    filtered_gdf.plot(ax=base, marker='o', markersize=10, column = 'observation_amount', cmap='viridis', legend=True, alpha=0.6)
    
    plt.title(f'Observations Heatmap for Year {year}')
    plt.show()
    
# Get the range of years in your data
min_year = gdf_halsbandparkiet['date'].dt.year.min()
max_year = gdf_halsbandparkiet['date'].dt.year.max()

# Create a slider
year_slider = widgets.IntSlider(value=min_year, min=min_year, max=max_year, step=1, description='Year:')

# Use interact to link the slider to the plot function
interact(plot_heatmap, year=year_slider)




interactive(children=(IntSlider(value=1971, description='Year:', max=2024, min=1971), Output()), _dom_classes=…

<function __main__.plot_heatmap(year)>

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns

# Plot heatmap
plt.figure(figsize=(10, 6))
sns.kdeplot(x=gdf_halsbandparkiet.geometry.x, y=gdf_halsbandparkiet.geometry.y, cmap="Reds", fill=True, levels=50, alpha=0.7)

# plt.scatter(x, y, s=5, color="black", alpha=0.5)  # Overlay points
plt.title("Spatial Density Heatmap")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.show()